In [5]:
from transformers import BertTokenizer
#from sklearn.datasets import fetch_20newsgroups #encountered issues SSLCertificationError with this 
from datasets import load_dataset
import os
import chardet

In [6]:
import pandas as pd
import re
from bs4 import BeautifulSoup

In [23]:
#Methods

def parse_html(text):
    "removes hyperlink from a piece of text"
    
    soup = BeautifulSoup(text, "html.parser")
    parsed_text = soup.get_text()
    return parsed_text


def preproc(text):

    refined_text = re.sub("(\n)", " ", text) #removes python newline 
    refined_text = re.sub("\w+(\')", "" , refined_text) #removes backslash
    refined_text = re.sub("(\s+)", " ", refined_text) #removes whitespaces
    refined_text = re.sub("(&amp|>+|-+)", " ", refined_text)
    refined_text = re.sub("(\d+)", "", refined_text) #removes digits
    refined_text = re.sub("\W*(@)", "", refined_text) #removes emails 

    #refined_text = re.sub("[>|<-\\", " ", )

    return refined_text

def prepare_corpus(path):
    """BY date"""
    corpus = []; temp = []
    for topic in os.listdir(path):
        subfolder = path + '/' + topic
        for doc in os.listdir(subfolder):
            file = subfolder + '/' + doc
            with open(file, 'r', encoding='utf-8', errors= 'ignore') as t:
                corpus.append((preproc(" ".join(t.readlines())),topic),)
    return corpus


def preproc_newsgroup(text):

    refined_text = re.sub("(\n)", " ", text)
    refined_text = re.sub("(\s+)", " ", refined_text)
    refined_text = re.sub("(&amp)", " ", refined_text)

    return

def check_length(text):
    return

def extra_long_documents(text):
    return 

In [8]:
hyperpartisan_dataset = load_dataset("hyperpartisan_news_detection", "byarticle") #Not stripped of headers and footers
hyperpartisan_dataset.keys()

#change to bypublisher because byarticle has no test set.
article_df = pd.DataFrame(hyperpartisan_dataset['train'])
#article_test_df = pd.DataFrame(hyperpartisan_dataset['test'])

article_text = []

#
train_text_list = list(article_df['text'])
train_tags_list = list(article_df['hyperpartisan'])
train_article_title = list(article_df['title'])

#
#test_text_list = list(article_test_df['text'])
#test_tags_list = list(article_test_df['hyperpartisan'])
#test_article_title = list(article_test_df['title'])

for item in train_text_list:
    article_text.append(preproc(parse_html(item)))
    
len(article_text)

Reusing dataset hyperpartisan_news_detection (/Users/max/.cache/huggingface/datasets/hyperpartisan_news_detection/byarticle/1.0.0/7f4215b0474950ddf516e806400ab81d098b3da3b3a919a13cd1a4cf2c677012)
100%|██████████| 1/1 [00:00<00:00, 590.75it/s]


645

In [81]:
newsgroup_path = "20news-bydate"
element = os.listdir(newsgroup_path)

newsgroup_train_path = newsgroup_path + '/' + element[1]
newsgroup_test_path = newsgroup_path + '/' + element[0]

newsgroup_train = prepare_corpus(newsgroup_train_path)
newsgroup_test = prepare_corpus(newsgroup_test_path)

len(newsgroup_train), newsgroup_train[0], newsgroup_train[0][0]

X_train_newsgroup_text, y_label = zip(*newsgroup_train)
X_test_newsgroup_text, y_label = zip(*newsgroup_test)

len(X_train_newsgroup_text),len(X_test_newsgroup_text)

(11314, 7532)

NAIVE BAYES EXPLORATION

In [71]:
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

Modelled the conversation with a bag of words approach

In [78]:
X_train_newsgroup_text[2000]

'From: perrydsinc.com (Jim Perry) Subject: Re: [soc.motss, et al.] "Princeton axes matching funds for Boy Scouts" Article I.D.: dsi.pqskINNhi Distribution: usa Organization: Decision Support Inc. Lines:  NNTP Posting Host: dsi.dsinc.com In article <Aprmidway.uchicago.edu  shoumidway.uchicago.edu writes:  In article <pidhINNubdsi.dsinc.com  perrydsinc.com (Jim Perry) writes:  Bigots never concede that their bigotry is irrational; it  is other people who determine that by examining their arguments.  [...]  No! I expected it! ve set yourself up a wonderful little  world where a bigot is whomever you say it is. This is very  comfortable for you imagine, never having to entertain an  argument against your belief system. Simply accuse the person  making of being a bigot. Well, this particular thread of vituperation slopped its venom over into alt.atheism, where we spend most of our time entertaining arguments against our belief system, without resorting to accusing others of bigotry. s somew

In [79]:
#from nltk.stem import WordNetLemmatizer
#from nltk.stem import PorterStemmer

#lemmatize = WordNetLemmatizer()
#stemmer = PorterStemmer()


def extract_features(text, method = "Tfidf"):
    """Count represents the number of features to be chosen from tfidf while text represents text data
    vectorizer is either count_vectorizer or tfidfvectorizer"""
    
    if method == "Tfidf":
        vectorizer = TfidfVectorizer()
    elif method == "Count":
        vectorizer = CountVectorizer()
    else:
        print("wrong method")

    vec = vectorizer.fit_transform(text)
    X_features = vectorizer.get_feature_names()
    print(vec.shape)
    return vec, X_features

vec, feat = extract_features(X_train_newsgroup_text, "Count")

#Filter features to have consistent features

(11314, 105178)


/Users/max/Desktop/Dissertation/implementations/Disso-COLD/proj_env/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [77]:
feat

['__',
 '___',
 '____',
 '_____',
 '______',
 '_______',
 '________',
 '_________',
 '__________',
 '_____________',
 '_______________',
 '________________',
 '_________________',
 '___________________',
 '_____________________',
 '________________________',
 '___________________________',
 '________________________________',
 '___________________________________',
 '_____________________________________________',
 '______________________________________________',
 '________________________________________________________',
 '__________________________________________________________',
 '_______________________________________________________________',
 '________________________________________________________________',
 '_________________________________________________________________',
 '___________________________________________________________________',
 '____________________________________________________________________',
 '_____________________________________________________

In [47]:
MNB = MultinomialNB()
MNB.fit(vec,y_label)




(11314,)

In [ ]:
acc = 0


for b,c in zip(A,y_label):
    if b ==c :
        acc +1 
    else:
        pass

(acc/len(y_label))*100

SVM

In [59]:
set(y_label[:2000])

{'alt.atheism', 'comp.sys.mac.hardware', 'rec.autos', 'talk.politics.mideast'}

In [80]:
from sklearn import svm
svc_classifier = svm.SVC(decision_function_shape='ovo')
svc_classifier.fit(vec,y_label)

SVC(decision_function_shape='ovo')

In [82]:
test_vec,test_features = extract_features(X_test_newsgroup_text)
svc_classifier.predict(test_vec)


(7532, 78590)


/Users/max/Desktop/Dissertation/implementations/Disso-COLD/proj_env/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


ValueError: X has 78590 features, but SVC is expecting 105178 features as input.